In [9]:
import netCDF4
import numpy as np
import pandas as pd
from scipy.interpolate import RegularGridInterpolator

# Pfad zur deiner .nc-Datei


# Name der Variablen, die die Maskendaten enthält (musst du eventuell anpassen)
mask_var_name = "lwe_thickness"

# Ziel-Grid-Dimensionen (5x5 Grad)
target_lon_start = -141.0
target_lon_end = -52.0
target_lat_start = 41.0
target_lat_end = 83.0
n_lon_target = int((target_lon_end - target_lon_start) / 5) + 1
n_lat_target = int((target_lat_end - target_lat_start) / 5) + 1

# Dateiname für die Ausgabe-CSV-Datei
output_csv_file = 'Mascons_Canada_grid_5x5.csv'

mascons = netCDF4.Dataset(nc_file_path, 'r')

#print all the variables

for var in mascons.variables:
    print(var)

# get the shape of the variables
print(mascons.variables['lwe_thickness'].shape)
print(mascons.variables['lon'].shape)
print(mascons.variables['lat'].shape)

import xarray as xr
import numpy as np
from scipy.interpolate import RectBivariateSpline
import pandas as pd

def interpolate_grace_to_grid(nc_file, resolution):
    """
    Interpolates GRACE LWE data to a coarser grid using spline interpolation.

    Args:
        nc_file (str): Path to the GRACE .nc file.
        resolution (float): The resolution of the target grid (e.g., 5 for a 5°x5° grid).

    Returns:
        pandas.DataFrame: A DataFrame containing the interpolated LWE thickness
                          for each grid cell and timestamp.
    """
    try:
        ds = xr.open_dataset(nc_file)
        lwe = ds['lwe_thickness']
        lat = ds['lat']
        lon = ds['lon']
        time = ds['time']
    except FileNotFoundError:
        print(f"Error: File not found at {nc_file}")
        return None
    except KeyError as e:
        print(f"Error: Variable not found in the NetCDF file: {e}")
        return None

    # Define the boundaries and centroids of the target grid
    lon_edges = np.arange(lon.min().item(), lon.max().item() + resolution, resolution)
    lat_edges = np.arange(lat.min().item(), lat.max().item() + resolution, resolution)
    lon_centroids = lon_edges[:-1] + resolution / 2
    lat_centroids = lat_edges[:-1] + resolution / 2

    interpolated_data = []

    for t_idx in range(len(time)):
        lwe_slice = lwe[t_idx, :, :].values

        # Create a spline interpolator for the current timestamp
        interpolator = RectBivariateSpline(lat, lon, lwe_slice)

        for lat_c in lat_centroids:
            for lon_c in lon_centroids:
                # Ensure the centroid is within the original data bounds for interpolation
                if (lat.min().item() <= lat_c <= lat.max().item() and
                        lon.min().item() <= lon_c <= lon.max().item()):
                    interpolated_value = interpolator(lat_c, lon_c, grid=False)
                    interpolated_data.append({
                        'time': pd.to_datetime(time[t_idx].item()),
                        'lon_centroid': lon_c,
                        'lat_centroid': lat_c,
                        'lwe_thickness': interpolated_value
                    })

    return pd.DataFrame(interpolated_data)

# --- Example Usage ---
if __name__ == "__main__":
    nc_file_path = '../../Data/JPL_Mascons_Canada_Interpolated_spline_modes2-5.nc'  
    grid_resolution = 5  # Define the desired grid resolution (e.g., 5 degrees)

    interpolated_df = interpolate_grace_to_grid(nc_file_path, grid_resolution)

    if interpolated_df is not None:
        output_csv_path = f'interpolated_lwe_grid_resolution{grid_resolution}.csv'
        interpolated_df.to_csv(output_csv_path, index=False)
        print(f"Interpolated data saved to {output_csv_path}")



lat
lon
time
lwe_thickness
crs
(272, 50, 160)
(160,)
(50,)
Interpolated data saved to interpolated_lwe_grid_resolution5.csv
